# Introduction

## Purpose of this notebook

This Colab notebook serves as a personal workbook to guide you (and me) through coding something from scratch.

I like to try to code some of Andrej Karpathy's 'Zero-to-Hero' projects from scratch every now and again.

By following this workbook, you can regularly practice and internalize the process of building from the ground up.

Each time you start a new Colab, simply copy over the instructions cell and build upon it.

## Purpose of this workbook

- This workbook only covers the 'naive' tokenizer built during the first hour of the video.

- Step-by-step intructions for coding this 'naive' tokenizer from scratch.

- Delete the contents of the code cell and re-write from scratch following the 'list of instructions'.

- Less than 50 lines of code.

- Once you have mastered the basics, then move on to the [Andrej Karpathy's recommended exercises](https://github.com/karpathy/minbpe/blob/master/exercise.md)


# Instructions

In [ ]:
# 'Naive' Byte Pair Encoding (BPE) Tokenizer Implementation

# Aim:
# - Train a tokenizer to efficiently encode and decode text using byte pair encoding.

# Artifacts from Training:
# - merges: Dictionary storing the merge rules learned during training.
# - vocab: Mapping from token id to bytes object for that token, updated with merges.

# Functions:
# - decode(ids): Convert token ids back to text.
# - encode(text): Convert text to token ids using the learned merges.

#-----------------------------------

# Minimal Flow to Code from Scratch:

# 1. Training the Tokenizer
# - Encode the text for training.
# - get_stats(): Find the frequency of each byte pair in the input.
# - Identify the most frequent byte pair (top_pair).
# - merge(): Replace occurrences of the top pair with a new token.
# - Loop to repeat the process and build merges.

# 2. Initialize Vocabulary
# - Create initial vocab mapping from token id to bytes object for that token.
# - Update vocab with merges.

# 3. Decoding
# - decode(ids): Convert token ids back to text.

# 4. Encoding
# - encode(text): Convert text to token ids using the learned merges.


# Training text

In [8]:
def get_long_string():
    return """A Programmer’s Introduction to Unicode March 3, 2017 · Coding · 22 Comments  Ｕｎｉｃｏｄｅ! 🅤🅝🅘🅒🅞🅓🅔‽ 🇺\u200c🇳\u200c🇮\u200c🇨\u200c🇴\u200c🇩\u200c🇪! 😄 The very name strikes fear and awe into the hearts of programmers worldwide. We all know we ought to “support Unicode” in our software (whatever that means—like using wchar_t for all the strings, right?). But Unicode can be abstruse, and diving into the thousand-page Unicode Standard plus its dozens of supplementary annexes, reports, and notes can be more than a little intimidating. I don’t blame programmers for still finding the whole thing mysterious, even 30 years after Unicode’s inception.  A few months ago, I got interested in Unicode and decided to spend some time learning more about it in detail. In this article, I’ll give an introduction to it from a programmer’s point of view.  I’m going to focus on the character set and what’s involved in working with strings and files of Unicode text. However, in this article I’m not going to talk about fonts, text layout/shaping/rendering, or localization in detail—those are separate issues, beyond my scope (and knowledge) here.  Diversity and Inherent Complexity The Unicode Codespace Codespace Allocation Scripts Usage Frequency Encodings UTF-8 UTF-16 Combining Marks Canonical Equivalence Normalization Forms Grapheme Clusters And More… Diversity and Inherent Complexity As soon as you start to study Unicode, it becomes clear that it represents a large jump in complexity over character sets like ASCII that you may be more familiar with. It’s not just that Unicode contains a much larger number of characters, although that’s part of it. Unicode also has a great deal of internal structure, features, and special cases, making it much more than what one might expect a mere “character set” to be. We’ll see some of that later in this article.  When confronting all this complexity, especially as an engineer, it’s hard not to find oneself asking, “Why do we need all this? Is this really necessary? Couldn’t it be simplified?”  However, Unicode aims to faithfully represent the entire world’s writing systems. The Unicode Consortium’s stated goal is “enabling people around the world to use computers in any language”. And as you might imagine, the diversity of written languages is immense! To date, Unicode supports 135 different scripts, covering some 1100 languages, and there’s still a long tail of over 100 unsupported scripts, both modern and historical, which people are still working to add.  Given this enormous diversity, it’s inevitable that representing it is a complicated project. Unicode embraces that diversity, and accepts the complexity inherent in its mission to include all human writing systems. It doesn’t make a lot of trade-offs in the name of simplification, and it makes exceptions to its own rules where necessary to further its mission.  Moreover, Unicode is committed not just to supporting texts in any single language, but also to letting multiple languages coexist within one text—which introduces even more complexity.  Most programming languages have libraries available to handle the gory low-level details of text manipulation, but as a programmer, you’ll still need to know about certain Unicode features in order to know when and how to apply them. It may take some time to wrap your head around it all, but don’t be discouraged—think about the billions of people for whom your software will be more accessible through supporting text in their language. Embrace the complexity!  The Unicode Codespace Let’s start with some general orientation. The basic elements of Unicode—its “characters”, although that term isn’t quite right—are called code points. Code points are identified by number, customarily written in hexadecimal with the prefix “U+”, such as U+0041 “A” latin capital letter a or U+03B8 “θ” greek small letter theta. Each code point also has a short name, and quite a few other properties, specified in the Unicode Character Database.  The set of all possible code points is called the codespace. The Unicode codespace consists of 1,114,112 code points. However, only 128,237 of them—about 12% of the codespace—are actually assigned, to date. There’s plenty of room for growth! Unicode also reserves an additional 137,468 code points as “private use” areas, which have no standardized meaning and are available for individual applications to define for their own purposes.  Codespace Allocation To get a feel for how the codespace is laid out, it’s helpful to visualize it. Below is a map of the entire codespace, with one pixel per code point. It’s arranged in tiles for visual coherence; each small square is 16×16 = 256 code points, and each large square is a “plane” of 65,536 code points. There are 17 planes altogether.  Map of the Unicode codespace (click to zoom)  White represents unassigned space. Blue is assigned code points, green is private-use areas, and the small red area is surrogates (more about those later). As you can see, the assigned code points are distributed somewhat sparsely, but concentrated in the first three planes.  Plane 0 is also known as the “Basic Multilingual Plane”, or BMP. The BMP contains essentially all the characters needed for modern text in any script, including Latin, Cyrillic, Greek, Han (Chinese), Japanese, Korean, Arabic, Hebrew, Devanagari (Indian), and many more.  (In the past, the codespace was just the BMP and no more—Unicode was originally conceived as a straightforward 16-bit encoding, with only 65,536 code points. It was expanded to its current size in 1996. However, the vast majority of code points in modern text belong to the BMP.)  Plane 1 contains historical scripts, such as Sumerian cuneiform and Egyptian hieroglyphs, as well as emoji and various other symbols. Plane 2 contains a large block of less-common and historical Han characters. The remaining planes are empty, except for a small number of rarely-used formatting characters in Plane 14; planes 15–16 are reserved entirely for private use.  Scripts Let’s zoom in on the first three planes, since that’s where the action is:  Map of scripts in Unicode planes 0–2 (click to zoom)  This map color-codes the 135 different scripts in Unicode. You can see how Han () and Korean () take up most of the range of the BMP (the left large square). By contrast, all of the European, Middle Eastern, and South Asian scripts fit into the first row of the BMP in this diagram.  Many areas of the codespace are adapted or copied from earlier encodings. For example, the first 128 code points of Unicode are just a copy of ASCII. This has clear benefits for compatibility—it’s easy to losslessly convert texts from smaller encodings into Unicode (and the other direction too, as long as no characters outside the smaller encoding are used).  Usage Frequency One more interesting way to visualize the codespace is to look at the distribution of usage—in other words, how often each code point is actually used in real-world texts. Below is a heat map of planes 0–2 based on a large sample of text from Wikipedia and Twitter (all languages). Frequency increases from black (never seen) through red and yellow to white.  Heat map of code point usage frequency in Unicode planes 0–2 (click to zoom)  You can see that the vast majority of this text sample lies in the BMP, with only scattered usage of code points from planes 1–2. The biggest exception is emoji, which show up here as the several bright squares in the bottom row of plane 1.  Encodings We’ve seen that Unicode code points are abstractly identified by their index in the codespace, ranging from U+0000 to U+10FFFF. But how do code points get represented as bytes, in memory or in a file?  The most convenient, computer-friendliest (and programmer-friendliest) thing to do would be to just store the code point index as a 32-bit integer. This works, but it consumes 4 bytes per code point, which is sort of a lot. Using 32-bit ints for Unicode will cost you a bunch of extra storage, memory, and performance in bandwidth-bound scenarios, if you work with a lot of text.  Consequently, there are several more-compact encodings for Unicode. The 32-bit integer encoding is officially called UTF-32 (UTF = “Unicode Transformation Format”), but it’s rarely used for storage. At most, it comes up sometimes as a temporary internal representation, for examining or operating on the code points in a string.  Much more commonly, you’ll see Unicode text encoded as either UTF-8 or UTF-16. These are both variable-length encodings, made up of 8-bit or 16-bit units, respectively. In these schemes, code points with smaller index values take up fewer bytes, which saves a lot of memory for typical texts. The trade-off is that processing UTF-8/16 texts is more programmatically involved, and likely slower.  UTF-8 In UTF-8, each code point is stored using 1 to 4 bytes, based on its index value.  UTF-8 uses a system of binary prefixes, in which the high bits of each byte mark whether it’s a single byte, the beginning of a multi-byte sequence, or a continuation byte; the remaining bits, concatenated, give the code point index. This table shows how it works:  UTF-8 (binary)\tCode point (binary)\tRange 0xxxxxxx\txxxxxxx\tU+0000–U+007F 110xxxxx 10yyyyyy\txxxxxyyyyyy\tU+0080–U+07FF 1110xxxx 10yyyyyy 10zzzzzz\txxxxyyyyyyzzzzzz\tU+0800–U+FFFF 11110xxx 10yyyyyy 10zzzzzz 10wwwwww\txxxyyyyyyzzzzzzwwwwww\tU+10000–U+10FFFF A handy property of UTF-8 is that code points below 128 (ASCII characters) are encoded as single bytes, and all non-ASCII code points are encoded using sequences of bytes 128–255. This has a couple of nice consequences. First, any strings or files out there that are already in ASCII can also be interpreted as UTF-8 without any conversion. Second, lots of widely-used string programming idioms—such as null termination, or delimiters (newlines, tabs, commas, slashes, etc.)—will just work on UTF-8 strings. ASCII bytes never occur inside the encoding of non-ASCII code points, so searching byte-wise for a null terminator or a delimiter will do the right thing.  Thanks to this convenience, it’s relatively simple to extend legacy ASCII programs and APIs to handle UTF-8 strings. UTF-8 is very widely used in the Unix/Linux and Web worlds, and many programmers argue UTF-8 should be the default encoding everywhere.  However, UTF-8 isn’t a drop-in replacement for ASCII strings in all respects. For instance, code that iterates over the “characters” in a string will need to decode UTF-8 and iterate over code points (or maybe grapheme clusters—more about those later), not bytes. When you measure the “length” of a string, you’ll need to think about whether you want the length in bytes, the length in code points, the width of the text when rendered, or something else.  UTF-16 The other encoding that you’re likely to encounter is UTF-16. It uses 16-bit words, with each code point stored as either 1 or 2 words.  Like UTF-8, we can express the UTF-16 encoding rules in the form of binary prefixes:  UTF-16 (binary)\tCode point (binary)\tRange xxxxxxxxxxxxxxxx\txxxxxxxxxxxxxxxx\tU+0000–U+FFFF 110110xxxxxxxxxx 110111yyyyyyyyyy\txxxxxxxxxxyyyyyyyyyy + 0x10000\tU+10000–U+10FFFF A more common way that people talk about UTF-16 encoding, though, is in terms of code points called “surrogates”. All the code points in the range U+D800–U+DFFF—or in other words, the code points that match the binary prefixes 110110 and 110111 in the table above—are reserved specifically for UTF-16 encoding, and don’t represent any valid characters on their own. They’re only meant to occur in the 2-word encoding pattern above, which is called a “surrogate pair”. Surrogate code points are illegal in any other context! They’re not allowed in UTF-8 or UTF-32 at all.  Historically, UTF-16 is a descendant of the original, pre-1996 versions of Unicode, in which there were only 65,536 code points. The original intention was that there would be no different “encodings”; Unicode was supposed to be a straightforward 16-bit character set. Later, the codespace was expanded to make room for a long tail of less-common (but still important) Han characters, which the Unicode designers didn’t originally plan for. Surrogates were then introduced, as—to put it bluntly—a kludge, allowing 16-bit encodings to access the new code points.  Today, Javascript uses UTF-16 as its standard string representation: if you ask for the length of a string, or iterate over it, etc., the result will be in UTF-16 words, with any code points outside the BMP expressed as surrogate pairs. UTF-16 is also used by the Microsoft Win32 APIs; though Win32 supports either 8-bit or 16-bit strings, the 8-bit version unaccountably still doesn’t support UTF-8—only legacy code-page encodings, like ANSI. This leaves UTF-16 as the only way to get proper Unicode support in Windows. (Update: in Win10 version 1903, they finally added UTF-8 support to the 8-bit APIs! 😊)  By the way, UTF-16’s words can be stored either little-endian or big-endian. Unicode has no opinion on that issue, though it does encourage the convention of putting U+FEFF zero width no-break space at the top of a UTF-16 file as a byte-order mark, to disambiguate the endianness. (If the file doesn’t match the system’s endianness, the BOM will be decoded as U+FFFE, which isn’t a valid code point.)  Combining Marks In the story so far, we’ve been focusing on code points. But in Unicode, a “character” can be more complicated than just an individual code point!  Unicode includes a system for dynamically composing characters, by combining multiple code points together. This is used in various ways to gain flexibility without causing a huge combinatorial explosion in the number of code points.  In European languages, for example, this shows up in the application of diacritics to letters. Unicode supports a wide range of diacritics, including acute and grave accents, umlauts, cedillas, and many more. All these diacritics can be applied to any letter of any alphabet—and in fact, multiple diacritics can be used on a single letter.  If Unicode tried to assign a distinct code point to every possible combination of letter and diacritics, things would rapidly get out of hand. Instead, the dynamic composition system enables you to construct the character you want, by starting with a base code point (the letter) and appending additional code points, called “combining marks”, to specify the diacritics. When a text renderer sees a sequence like this in a string, it automatically stacks the diacritics over or under the base letter to create a composed character.  For example, the accented character “Á” can be expressed as a string of two code points: U+0041 “A” latin capital letter a plus U+0301 “◌́” combining acute accent. This string automatically gets rendered as a single character: “Á”.  Now, Unicode does also include many “precomposed” code points, each representing a letter with some combination of diacritics already applied, such as U+00C1 “Á” latin capital letter a with acute or U+1EC7 “ệ” latin small letter e with circumflex and dot below. I suspect these are mostly inherited from older encodings that were assimilated into Unicode, and kept around for compatibility. In practice, there are precomposed code points for most of the common letter-with-diacritic combinations in European-script languages, so they don’t use dynamic composition that much in typical text.  Still, the system of combining marks does allow for an arbitrary number of diacritics to be stacked on any base character. The reductio-ad-absurdum of this is Zalgo text, which works by ͖͟ͅr͞aṋ̫̠̖͈̗d͖̻̹óm̪͙͕̗̝ļ͇̰͓̳̫ý͓̥̟͍ ̕s̫t̫̱͕̗̰̼̘͜a̼̩͖͇̠͈̣͝c̙͍k̖̱̹͍͘i̢n̨̺̝͇͇̟͙ģ̫̮͎̻̟ͅ ̕n̼̺͈͞u̮͙m̺̭̟̗͞e̞͓̰̤͓̫r̵o̖ṷs҉̪͍̭̬̝̤ ̮͉̝̞̗̟͠d̴̟̜̱͕͚i͇̫̼̯̭̜͡ḁ͙̻̼c̲̲̹r̨̠̹̣̰̦i̱t̤̻̤͍͙̘̕i̵̜̭̤̱͎c̵s ͘o̱̲͈̙͖͇̲͢n͘ ̜͈e̬̲̠̩ac͕̺̠͉h̷̪ ̺̣͖̱ḻ̫̬̝̹ḙ̙̺͙̭͓̲t̞̞͇̲͉͍t̷͔̪͉̲̻̠͙e̦̻͈͉͇r͇̭̭̬͖,̖́ ̜͙͓̣̭s̘̘͈o̱̰̤̲ͅ ̛̬̜̙t̼̦͕̱̹͕̥h̳̲͈͝ͅa̦t̻̲ ̻̟̭̦̖t̛̰̩h̠͕̳̝̫͕e͈̤̘͖̞͘y҉̝͙ ̷͉͔̰̠o̞̰v͈͈̳̘͜er̶f̰͈͔ḻ͕̘̫̺̲o̲̭͙͠ͅw̱̳̺ ͜t̸h͇̭͕̳͍e̖̯̟̠ ͍̞̜͔̩̪͜ļ͎̪̲͚i̝̲̹̙̩̹n̨̦̩̖ḙ̼̲̼͢ͅ ̬͝s̼͚̘̞͝p͙̘̻a̙c҉͉̜̤͈̯̖i̥͡n̦̠̱͟g̸̗̻̦̭̮̟ͅ ̳̪̠͖̳̯̕a̫͜n͝d͡ ̣̦̙ͅc̪̗r̴͙̮̦̹̳e͇͚̞͔̹̫͟a̙̺̙ț͔͎̘̹ͅe̥̩͍ a͖̪̜̮͙̹n̢͉̝ ͇͉͓̦̼́a̳͖̪̤̱p̖͔͔̟͇͎͠p̱͍̺ę̲͎͈̰̲̤̫a̯͜r̨̮̫̣̘a̩̯͖n̹̦̰͎̣̞̞c̨̦̱͔͎͍͖e̬͓͘ ̤̰̩͙̤̬͙o̵̼̻̬̻͇̮̪f̴ ̡̙̭͓͖̪̤“̸͙̠̼c̳̗͜o͏̼͙͔̮r̞̫̺̞̥̬ru̺̻̯͉̭̻̯p̰̥͓̣̫̙̤͢t̳͍̳̖ͅi̶͈̝͙̼̙̹o̡͔n̙̺̹̖̩͝ͅ”̨̗͖͚̩.̯͓  A few other places where dynamic character composition shows up in Unicode:  Vowel-pointing notation in Arabic and Hebrew. In these languages, words are normally spelled with some of their vowels left out. They then have diacritic notation to indicate the vowels (used in dictionaries, language-teaching materials, children’s books, and such). These diacritics are expressed with combining marks.  A Hebrew example, with niqqud:\tאֶת דַלְתִּי הֵזִיז הֵנִיעַ, קֶטֶב לִשְׁכַּתִּי יָשׁוֹד Normal writing (no niqqud):\tאת דלתי הזיז הניע, קטב לשכתי ישוד Devanagari, the script used to write Hindi, Sanskrit, and many other South Asian languages, expresses certain vowels as combining marks attached to consonant letters. For example, “ह” + “\u200bि” = “हि” (“h” + “i” = “hi”). Korean characters stand for syllables, but they are composed of letters called jamo that stand for the vowels and consonants in the syllable. While there are code points for precomposed Korean syllables, it’s also possible to dynamically compose them by concatenating their jamo. For example, “ᄒ” + “ᅡ” + “ᆫ” = “한” (“h” + “a” + “n” = “han”). Canonical Equivalence In Unicode, precomposed characters exist alongside the dynamic composition system. A consequence of this is that there are multiple ways to express “the same” string—different sequences of code points that result in the same user-perceived characters. For example, as we saw earlier, we can express the character “Á” either as the single code point U+00C1, or as the string of two code points U+0041 U+0301.  Another source of ambiguity is the ordering of multiple diacritics in a single character. Diacritic order matters visually when two diacritics apply to the same side of the base character, e.g. both above: “ǡ” (dot, then macron) is different from “ā̇” (macron, then dot). However, when diacritics apply to different sides of the character, e.g. one above and one below, then the order doesn’t affect rendering. Moreover, a character with multiple diacritics might have one of the diacritics precomposed and others expressed as combining marks.  For example, the Vietnamese letter “ệ” can be expressed in five different ways:  Fully precomposed: U+1EC7 “ệ” Partially precomposed: U+1EB9 “ẹ” + U+0302 “◌̂” Partially precomposed: U+00EA “ê” + U+0323 “◌̣” Fully decomposed: U+0065 “e” + U+0323 “◌̣” + U+0302 “◌̂” Fully decomposed: U+0065 “e” + U+0302 “◌̂” + U+0323 “◌̣” Unicode refers to set of strings like this as “canonically equivalent”. Canonically equivalent strings are supposed to be treated as identical for purposes of searching, sorting, rendering, text selection, and so on. This has implications for how you implement operations on text. For example, if an app has a “find in file” operation and the user searches for “ệ”, it should, by default, find occurrences of any of the five versions of “ệ” above!  Normalization Forms To address the problem of “how to handle canonically equivalent strings”, Unicode defines several normalization forms: ways of converting strings into a canonical form so that they can be compared code-point-by-code-point (or byte-by-byte).  The “NFD” normalization form fully decomposes every character down to its component base and combining marks, taking apart any precomposed code points in the string. It also sorts the combining marks in each character according to their rendered position, so e.g. diacritics that go below the character come before the ones that go above the character. (It doesn’t reorder diacritics in the same rendered position, since their order matters visually, as previously mentioned.)  The “NFC” form, conversely, puts things back together into precomposed code points as much as possible. If an unusual combination of diacritics is called for, there may not be any precomposed code point for it, in which case NFC still precomposes what it can and leaves any remaining combining marks in place (again ordered by rendered position, as in NFD).  There are also forms called NFKD and NFKC. The “K” here refers to compatibility decompositions, which cover characters that are “similar” in some sense but not visually identical. However, I’m not going to cover that here.  Grapheme Clusters As we’ve seen, Unicode contains various cases where a thing that a user thinks of as a single “character” might actually be made up of multiple code points under the hood. Unicode formalizes this using the notion of a grapheme cluster: a string of one or more code points that constitute a single “user-perceived character”.  UAX #29 defines the rules for what, precisely, qualifies as a grapheme cluster. It’s approximately “a base code point followed by any number of combining marks”, but the actual definition is a bit more complicated; it accounts for things like Korean jamo, and emoji ZWJ sequences.  The main thing grapheme clusters are used for is text editing: they’re often the most sensible unit for cursor placement and text selection boundaries. Using grapheme clusters for these purposes ensures that you can’t accidentally chop off some diacritics when you copy-and-paste text, that left/right arrow keys always move the cursor by one visible character, and so on.  Another place where grapheme clusters are useful is in enforcing a string length limit—say, on a database field. While the true, underlying limit might be something like the byte length of the string in UTF-8, you wouldn’t want to enforce that by just truncating bytes. At a minimum, you’d want to “round down” to the nearest code point boundary; but even better, round down to the nearest grapheme cluster boundary. Otherwise, you might be corrupting the last character by cutting off a diacritic, or interrupting a jamo sequence or ZWJ sequence.  And More… There’s much more that could be said about Unicode from a programmer’s perspective! I haven’t gotten into such fun topics as case mapping, collation, compatibility decompositions and confusables, Unicode-aware regexes, or bidirectional text. Nor have I said anything yet about implementation issues—how to efficiently store and look-up data about the sparsely-assigned code points, or how to optimize UTF-8 decoding, string comparison, or NFC normalization. Perhaps I’ll return to some of those things in future posts.  Unicode is a fascinating and complex system. It has a many-to-one mapping between bytes and code points, and on top of that a many-to-one (or, under some circumstances, many-to-many) mapping between code points and “characters”. It has oddball special cases in every corner. But no one ever claimed that representing all written languages was going to be easy, and it’s clear that we’re never going back to the bad old days of a patchwork of incompatible encodings.  Further reading:  The Unicode Standard UTF-8 Everywhere Manifesto Dark corners of Unicode by Eevee ICU (International Components for Unicode)—C/C++/Java libraries implementing many Unicode algorithms and related things Python 3 Unicode Howto Google Noto Fonts—set of fonts intended to cover all assigned code points"""

# Code (delete this cell and re-write from scratch)

In [16]:
#-----------------------------------

text = get_long_string()
tokens = list(text.encode('utf-8'))

#-----------------------------------

def get_stats(ids):
    counts = {}
    for pair in zip(ids[:], ids[1:]):
        counts[pair] = counts.get(pair, 0) + 1
    return counts

stats = get_stats(tokens)
top_pair = max(stats, key=stats.get)


def merge(ids, pair, idx):
    newids = []
    i=0
    while(i<len(ids)):
        if (i<len(ids)-1 and ids[i]==pair[0] and ids[i+1]==pair[1]):
            newids.append(idx)
            i+=2
        else:
            newids.append(ids[i])
            i+=1
    return newids

#------------------------

vocab_size = 276
num_merges = vocab_size - 256

merges = {}
ids = list(tokens)

for i in range(num_merges):
    stats = get_stats(ids)
    pair = max(stats, key=stats.get)
    idx = i + 256
    merges[pair] = idx
    ids = merge(ids, pair, idx)

#------------------------

vocab = {idx: bytes([idx]) for idx in range(256)}
for (p0, p1), idx in merges.items():
    vocab[idx] = vocab[p0] + vocab[p1]

#------------------------

def decode(ids):
    tokens = b"".join(vocab[idx] for idx in ids)
    text = tokens.decode('utf-8', errors='replace')
    return text

def encode(text):
    tokens = list(text.encode('utf-8'))
    while(len(tokens)>=2):
        stats = get_stats(tokens)
        pair = max(stats, key=lambda p: merges.get(p, float('inf')))
        if pair not in merges:
            break
        else:
            idx = merges[pair]
            tokens = merge(tokens, pair, idx)
    return tokens

decode(encode('hello world!'))


'hello world!'

# Validation text

In [17]:
valtext = "Many common characters, including numerals, punctuation, and other symbols, are unified within the standard and are not treated as specific to any given writing system. Unicode encodes thousands of emoji, with the continued development thereof conducted by the Consortium as a part of the standard.[4] Moreover, the widespread adoption of Unicode was in large part responsible for the initial popularization of emoji outside of Japan. Unicode is ultimately capable of encoding more than 1.1 million characters."
valtext2 = decode(encode(valtext))
print(valtext2 == valtext)

True


# Notes

## What is a Bytes Object?

When you use the `encode("utf-8")` method on a string in Python, it converts the string into a bytes object. Here’s a detailed explanation of what happens and how the returned object relates to a string or a list:

### The `encode` Method

The `encode` method is used to convert a string (which is a sequence of characters) into a bytes object (which is a sequence of bytes). The method takes an encoding format as an argument, in this case, "utf-8".

```python
text = "Hello, world!"
encoded_text = text.encode("utf-8")
```

### What is a Bytes Object?

A bytes object is an immutable sequence of bytes. Each byte is represented by an integer in the range 0-255. In the context of UTF-8 encoding, the bytes object represents the string as a sequence of bytes according to the UTF-8 encoding scheme.

### Relationship to a String

- **String**: A string in Python is a sequence of Unicode characters. Each character can be represented by multiple bytes, especially for non-ASCII characters.
- **Bytes Object**: A bytes object is a sequence of bytes. When a string is encoded to bytes using UTF-8, each character in the string is converted into one or more bytes.

For example:
```python
text = "Hello"
encoded_text = text.encode("utf-8")
print(encoded_text)  # Output: b'Hello'
```

In this example, each character in the string "Hello" is represented by a single byte in UTF-8 because these are ASCII characters.

### Relationship to a List

A bytes object can be thought of as similar to a list of integers where each integer is a byte (0-255). However, unlike a list, a bytes object is immutable, meaning it cannot be changed after it is created.

You can convert a bytes object to a list of integers:
```python
encoded_text = text.encode("utf-8")
byte_list = list(encoded_text)
print(byte_list)  # Output: [72, 101, 108, 108, 111]
```

Here, each integer in the list represents the byte value of the corresponding character in the encoded string.

### Example with Non-ASCII Characters

For non-ASCII characters, UTF-8 uses multiple bytes to represent each character:
```python
text = "你好"
encoded_text = text.encode("utf-8")
print(encoded_text)  # Output: b'\xe4\xbd\xa0\xe5\xa5\xbd'
```

Here, the Chinese characters are represented by multiple bytes:
- '你' is represented by three bytes: `'\xe4\xbd\xa0'`
- '好' is represented by three bytes: `'\xe5\xa5\xbd'`

### Summary

- **String**: Sequence of Unicode characters.
- **Bytes Object**: Sequence of bytes, immutable, represents the encoded version of the string.
- **List**: While a bytes object is not a list, you can convert it to a list of integers representing each byte.

The `encode("utf-8")` method returns a bytes object that represents the original string in UTF-8 encoding. This bytes object is immutable and can be thought of as a sequence of byte values.

## `bytes()`

The `bytes()` function in Python is a built-in function that returns a new "bytes" object, which is an immutable sequence of bytes. This is often used in situations where binary data needs to be handled, such as when working with files, network protocols, or data encryption.

Here’s a detailed explanation of the `bytes()` function:

### Syntax
```python
bytes([source[, encoding[, errors]]])
```

### Parameters
- **source (optional)**: The source to initialize the bytes object. This can be:
  - A string (must also provide the `encoding` argument).
  - An iterable of integers in the range 0 <= x < 256.
  - An object implementing the buffer interface.
  - An integer, which specifies the size of the bytes object to create (initialized with null bytes).
  - If no source is provided, an empty bytes object is created.
- **encoding (optional)**: The name of the encoding to use if the source is a string.
- **errors (optional)**: The error handling scheme to use for encoding errors (e.g., 'strict', 'ignore', 'replace').

### Examples

#### 1. Creating an empty bytes object
```python
empty_bytes = bytes()
print(empty_bytes)  # Output: b''
```

#### 2. Creating a bytes object from an iterable of integers
```python
byte_array = bytes([65, 66, 67])
print(byte_array)  # Output: b'ABC'
```

#### 3. Creating a bytes object from a string with encoding
```python
string_bytes = bytes('hello', 'utf-8')
print(string_bytes)  # Output: b'hello'
```

#### 4. Creating a bytes object of a specified size
```python
size_bytes = bytes(5)
print(size_bytes)  # Output: b'\x00\x00\x00\x00\x00'
```

### Key Points
- **Immutability**: Bytes objects are immutable sequences, meaning once created, their contents cannot be changed.
- **Utility in Binary Data**: Bytes are particularly useful for representing binary data, such as the contents of files or network packets.
- **Compatibility with Buffer Protocol**: Bytes objects are compatible with Python's buffer protocol, allowing for efficient storage and manipulation of binary data.

### Practical Usage
#### Reading Binary Data from a File
```python
with open('example.bin', 'rb') as f:
    data = f.read()
    print(data)  # Output: binary data from the file as a bytes object
```

#### Converting Bytes to a String
```python
byte_data = b'hello'
string_data = byte_data.decode('utf-8')
print(string_data)  # Output: 'hello'
```

#### Handling Encoding and Decoding
```python
# Encoding a string to bytes
string_data = "Python"
encoded_data = string_data.encode('utf-8')
print(encoded_data)  # Output: b'Python'

# Decoding bytes to a string
decoded_data = encoded_data.decode('utf-8')
print(decoded_data)  # Output: 'Python'
```

The `bytes()` function is a versatile tool in Python for dealing with binary data, and understanding its usage is crucial for tasks involving file I/O, network communication, and data serialization.

## `ord()` function in Python

The `ord()` function in Python is a built-in function used to return the Unicode code point for a given character. The Unicode code point is an integer that uniquely identifies a character in the Unicode standard. Here are the main use cases of the `ord()` function along with examples:

### 1. **Character to Unicode Conversion**

The primary use of `ord()` is to convert a single character to its corresponding Unicode code point.

**Example:**
```python
char = 'A'
unicode_code_point = ord(char)
print(unicode_code_point)  # Output: 65
```

### 2. **Character Manipulation**

`ord()` can be used in conjunction with the `chr()` function (which converts a Unicode code point back to a character) to perform character manipulation. For example, shifting characters in the alphabet.

**Example:**
```python
# Shift character 'A' to the next character in the alphabet
char = 'A'
shifted_char = chr(ord(char) + 1)
print(shifted_char)  # Output: 'B'
```

### 3. **Sorting Characters**

When sorting characters, `ord()` can be used to compare the Unicode code points of characters, ensuring the correct order.

**Example:**
```python
chars = ['b', 'a', 'c']
sorted_chars = sorted(chars, key=lambda x: ord(x))
print(sorted_chars)  # Output: ['a', 'b', 'c']
```

### 4. **Character Range Checking**

`ord()` can help in checking if a character falls within a specific range, such as checking if a character is an uppercase letter, lowercase letter, or digit.

**Example:**
```python
char = 'G'
if ord('A') <= ord(char) <= ord('Z'):
    print(f'{char} is an uppercase letter')  # Output: G is an uppercase letter
```

### 5. **Encoding and Decoding**

In text processing and encoding tasks, `ord()` can be used to convert characters to their numeric representations for various encoding schemes.

**Example:**
```python
text = "Hello"
encoded_text = [ord(char) for char in text]
print(encoded_text)  # Output: [72, 101, 108, 108, 111]
```

### 6. **Custom Sorting and Filtering**

When working with strings, `ord()` can help in custom sorting and filtering operations based on Unicode code points.

**Example:**
```python
# Filter out non-alphabet characters from a string
text = "Hello123!"
filtered_text = ''.join([char for char in text if 'A' <= char <= 'Z' or 'a' <= char <= 'z'])
print(filtered_text)  # Output: "Hello"
```

### Summary

The `ord()` function is versatile and primarily used for:
- Converting characters to Unicode code points.
- Character manipulation and shifting.
- Sorting characters based on their Unicode values.
- Checking character ranges.
- Encoding and decoding text.
- Custom sorting and filtering operations.

These use cases demonstrate how `ord()` can be utilized in various scenarios involving character and string manipulation.

## Python's String `.encode()` Function

### Main Use Cases of Python's String `.encode()` Function

The `.encode()` function in Python is used to convert a string (which is in Unicode by default) into a bytes object, which can be stored and manipulated in various ways. Here are the main use cases:

1. **Data Transmission and Storage**:
   - Encoding strings is essential for transmitting data over networks or saving data to files. Different systems might use different encodings, so converting strings to a standardized encoding like UTF-8 ensures compatibility.

2. **Interoperability**:
   - Many external systems, APIs, and libraries expect data to be in a specific byte format. Encoding strings into bytes allows Python applications to interact with these systems correctly.

3. **Memory Efficiency**:
   - In some cases, storing data as bytes can be more memory-efficient than storing it as Unicode strings, especially when dealing with large datasets.

4. **Data Processing**:
   - Certain data processing tasks, such as encryption or compression, require data to be in a byte format. Encoding strings is the first step in these processes.

### Examples Contrasting "utf-8" to Other Encodings

Here are some examples showing how the `.encode()` function works with different encodings, highlighting the differences between UTF-8 and other encodings.

#### UTF-8 Encoding

UTF-8 is a variable-width character encoding capable of encoding all possible characters in Unicode. It is the most common encoding used on the web.

```python
text = "Hello, world! Привет, мир! 你好，世界！"

# Encoding the string using UTF-8
utf8_encoded = text.encode('utf-8')
print(utf8_encoded)
```

Output:
```
b'Hello, world! \xd0\x9f\xd1\x80\xd0\xb8\xd0\xb2\xd0\xb5\xd1\x82, \xd0\xbc\xd0\xb8\xd1\x80! \xe4\xbd\xa0\xe5\xa5\xbd\xef\xbc\x8c\xe4\xb8\x96\xe7\x95\x8c\xef\xbc\x81'
```

#### ASCII Encoding

ASCII is a character encoding standard for electronic communication, representing text in computers. It uses 7-bit binary numbers to represent characters and is limited to 128 characters.

```python
# Encoding the string using ASCII
try:
    ascii_encoded = text.encode('ascii')
except UnicodeEncodeError as e:
    print(f"Encoding Error: {e}")
```

Output:
```
Encoding Error: 'ascii' codec can't encode characters in position 13-33: ordinal not in range(128)
```

ASCII encoding fails because it cannot represent non-ASCII characters (like Cyrillic or Chinese characters).

#### ISO-8859-1 (Latin-1) Encoding

ISO-8859-1 is a single-byte encoding that can represent the first 256 Unicode characters. It is used for Western European languages.

```python
# Encoding the string using ISO-8859-1
try:
    latin1_encoded = text.encode('iso-8859-1')
except UnicodeEncodeError as e:
    print(f"Encoding Error: {e}")
```

Output:
```
Encoding Error: 'latin-1' codec can't encode characters in position 13-33: ordinal not in range(256)
```

Similar to ASCII, ISO-8859-1 fails because it cannot encode the Cyrillic and Chinese characters.

#### UTF-16 Encoding

UTF-16 is a character encoding capable of encoding all 1,112,064 valid character code points in Unicode using one or two 16-bit code units.

```python
# Encoding the string using UTF-16
utf16_encoded = text.encode('utf-16')
print(utf16_encoded)
```

Output:
```
b'\xff\xfeH\x00e\x00l\x00l\x00o\x00,\x00 \x00w\x00o\x00r\x00l\x00d\x00!\x00 \x00\x1f\x04@\x04B\x04>\x04B\x04<\x04@\x04A\x04,\x00 \x00>\x04B\x04A\x04@\x04!\x00 \x00`O}Y\x0cO\x16U'
```

#### Summary

- **UTF-8**: Variable-length, most widely used, supports all Unicode characters.
- **ASCII**: Single-byte, limited to 128 characters, fails with non-ASCII characters.
- **ISO-8859-1 (Latin-1)**: Single-byte, supports the first 256 Unicode characters, fails with non-Latin characters.
- **UTF-16**: Fixed-length (mostly), supports all Unicode characters, uses more memory compared to UTF-8 for ASCII characters.

Each encoding has its use case depending on the language and the specific application requirements. UTF-8 is generally preferred for its compatibility and efficiency with a wide range of characters.

## is `map(int, tokens)` necessary?

The line `tokens = list(tokens)` does not automatically convert the elements to integers.

### Original Code:

```python
tokens = text.encode("utf-8")  # raw bytes
tokens = list(map(int, tokens))  # convert to a list of integers in range 0..255 for convenience
```

### Step-by-Step Breakdown:

1. **Encoding the Text**:
    ```python
    tokens = text.encode("utf-8")
    ```
    - This converts the text string into a UTF-8 encoded byte sequence.
    - For example, the string `"hello"` would be encoded to the bytes sequence `b'hello'`.

2. **Mapping to Integers**:
    ```python
    tokens = list(map(int, tokens))
    ```
    - This converts each byte in the byte sequence to its corresponding integer value.
    - For example, the byte sequence `b'hello'` would be converted to the list `[104, 101, 108, 108, 111]`.

### What Happens with `list(tokens)`:

If you only use `list(tokens)` without `map(int, tokens)`, it will convert the byte sequence into a list of bytes, not integers. Here’s an example to illustrate the difference:

#### Example with `map(int, tokens)`:

```python
text = "hello"
tokens = text.encode("utf-8")  # b'hello'
tokens = list(map(int, tokens))  # [104, 101, 108, 108, 111]
```

- `text.encode("utf-8")` produces the byte sequence `b'hello'`.
- `list(map(int, tokens))` converts each byte to its corresponding integer value: `[104, 101, 108, 108, 111]`.

#### Example with `list(tokens)`:

```python
text = "hello"
tokens = text.encode("utf-8")  # b'hello'
tokens = list(tokens)  # [104, 101, 108, 108, 111]
```

- `text.encode("utf-8")` produces the byte sequence `b'hello'`.
- `list(tokens)` without `map(int, tokens)` results in `[104, 101, 108, 108, 111]` which is essentially the same since Python's `bytes` type directly converts to integers when iterated over in a list context.

### Conclusion:

While it may seem redundant, in Python 3, converting the `bytes` to a list using `list(tokens)` does indeed produce the same result as `list(map(int, tokens))` because each element in a `bytes` object is already an integer (0-255). Therefore, both lines of code achieve the same result of converting the byte sequence to a list of integers.

So, strictly speaking, `tokens = list(tokens)` would work correctly and is sufficient to get the list of integers, making the explicit `map(int, tokens)` redundant in this specific context.

## Why is the starting vocabulary size 256?

The starting vocabulary size of 256 in the provided byte pair encoding (BPE) implementation is likely chosen for several reasons related to the characteristics of the input data and the efficiency of encoding and processing.

1. **Byte-Level Encoding:**
   - The number 256 is significant because it represents the total number of unique byte values in an 8-bit byte. This suggests that the initial tokens are likely individual bytes or characters encoded in a single byte. By starting with 256, the algorithm covers all possible byte values, ensuring that every possible input character is represented initially.

2. **Simplicity and Universality:**
   - Using byte-level encoding simplifies the initial setup. Every byte or character can be uniquely identified without requiring any further tokenization or preprocessing. This is particularly useful for handling a wide range of text inputs, including different languages and special characters.

3. **Efficiency in Data Compression:**
   - BPE aims to merge the most frequent pairs of tokens iteratively to reduce the overall length of the tokenized sequence. Starting with a base of 256 tokens provides a granular and comprehensive starting point, allowing the algorithm to effectively find and merge the most frequent pairs, optimizing for compression and efficiency.

4. **Compatibility with Encoding Standards:**
   - Many text encoding standards, such as ASCII and extended ASCII, are based on 256 unique character representations. By aligning the starting vocabulary size with these standards, the implementation ensures compatibility with a broad range of text data.

5. **Memory and Computational Considerations:**
   - Managing a vocabulary of 256 tokens is computationally feasible and efficient. It provides a balance between having a sufficiently granular initial set of tokens and maintaining manageable memory and computational overhead. As the algorithm proceeds with merging, the vocabulary size increases, but starting with 256 ensures the process begins with a manageable and standardized set.

In summary, the starting vocabulary size of 256 is chosen to align with the byte-level representation of characters, ensuring comprehensive initial coverage, compatibility with standard text encodings, and efficient processing. This approach lays a solid foundation for the BPE algorithm to iteratively merge frequent pairs and optimize the encoding for the target vocabulary size.

## How to fold a colab code cell

In Google Colab, you have a few options to hide or minimize a very long string or large code cell to make your notebook more manageable:

### 1. **Folding the Cell:**
Google Colab allows you to fold the cells to hide the content. To fold a cell, click the small arrow to the left of the cell number. This will collapse the cell, hiding its contents. (You might need to enable code folding in settings)

### 2. **Using a Markdown Cell:**
You can place the long string in a Markdown cell and then hide it using HTML tags. For example:

```markdown
<details>
<summary>Click to see the long string</summary>
<p>

```
<Your very long string goes here>

```markdown
</p>
</details>
```

When you run the Markdown cell, it will show a clickable "Click to see the long string" that will expand to show the hidden content.

### 3. **Using a Python Function:**
You can store the long string in a Python function or a variable, which you can then call when needed. For example:

```python
def get_long_string():
    return """Your very long string here"""
```

This keeps the cell clean, and you can print or use the string by calling `get_long_string()`.

### 4. **IPython Display Functions:**
You can use IPython's display functions to hide the content and only display it when needed:

```python
from IPython.display import display, HTML

long_string = "Your very long string here"

display(HTML(f"<details><summary>Click to see the long string</summary><p>{long_string}</p></details>"))
```

This way, you can hide the long string behind a clickable summary.

### 5. **Code Snippets:**
If you frequently use this long string, you might consider using code snippets. While Colab itself doesn’t have a built-in snippet manager, you can use browser extensions or third-party tools to manage code snippets that you can easily insert into your notebook.

### 6. **Custom JavaScript:**
For advanced users, you can use custom JavaScript to hide or show elements dynamically, but this is less commonly used and requires familiarity with JavaScript and the IPython notebook environment.

Choose the method that best fits your workflow and needs. Using Markdown and IPython display functions are usually the most convenient and readable ways to manage long strings in a Colab notebook.